In [20]:
'''
Figure out why there is a difference in the html files
'''

In [2]:
import urllib
import sys
from bs4 import BeautifulSoup

In [63]:
url = "https://www.reddit.com/r/spaceporn"
page = urllib.request.urlopen(url)

HTTPError: HTTP Error 429: Too Many Requests

In [12]:
f = open("spaceporn.txt", 'r')
soup = BeautifulSoup(f.read(), "html.parser")

In [19]:
import re
from constants import DEFAULT, TOL
anc_pass = []

for anc in soup.find_all('a'):
    res = re.search('\d{4}\s*[x]\s*\d{4}', str(anc)) # look for "0000 x 0000" tags
    if res:
        anc_pass.append(anc)
        w = int(res.group()[:4])
        h = int(res.group()[-4:])
        if abs(DEFAULT - w / h) < TOL:
            pass

# clues: data-outbound-url, data-href-url
print(len(anc_pass))
anc_pass

23


[<a class="title may-blank loggedin outbound" data-event-action="title" data-href-url="https://cdn.eso.org/images/large/eso0905a.jpg" data-outbound-expiration="1532671164000" data-outbound-url="https://out.reddit.com/t3_9275r8?url=https%3A%2F%2Fcdn.eso.org%2Fimages%2Flarge%2Feso0905a.jpg&amp;token=AQAAvLRaWxEEgGpcJVI-zEk-L2wo9JqkjbIZfPAMx1LoEirNhtsp&amp;app_name=reddit.com" href="https://cdn.eso.org/images/large/eso0905a.jpg" rel="" tabindex="1">The Carina Nebula [8408 x 8337]</a>,
 <a class="title may-blank loggedin " data-event-action="title" data-href-url="/r/spaceporn/comments/9299lt/starstruck_the_milky_way_rises_over_quabbin/" data-inbound-url="/r/spaceporn/comments/9299lt/starstruck_the_milky_way_rises_over_quabbin/?utm_content=title&amp;utm_medium=hot&amp;utm_source=reddit&amp;utm_name=spaceporn" href="/r/spaceporn/comments/9299lt/starstruck_the_milky_way_rises_over_quabbin/" rel="" tabindex="1">Starstruck: The Milky Way rises over Quabbin Reservoir. Possibly one of the darkest

In [10]:
def astrowall():
    status, folder = gen_folder()

    print("Scraping..." if status else "You already scraped today.")

    if status:
        anchors  = scrapity_scroopity()
        img_urls = filter_links(anchors)
        save_imgs(img_urls)
        
    print("Changing wallpaper...")
    set_wall(folder)

In [1]:
def gen_folder():
    '''
    Returns tuple of true if the folder of the week already exists and the folder name.
    '''
    import datetime as dt
    import os
    
    PIC_PATH = "pictures/"
    os.chdir(PIC_PATH)
    status = True
    
    if not os.path.isdir(dt.date.today().strftime("%d%m%Y")):
        status = False
        os.makedirs(new_folder)
    
    return status, new_folder

In [ ]:
def scrapity_scroopity():
    '''
    Run query and return a list of anchor tags
    
    Notes:
    https://www.analyticsvidhya.com/blog/2015/10/beginner-guide-web-scraping-beautiful-soup-python/
    '''
    import requests
    from bs4 import BeautifulSoup
    
    url = "https://www.reddit.com/r/spaceporn"
    page = requests.get(url)

In [ ]:
def filter_links(anchors):
    '''
    Parse the given list of anchor tags and return a list of urls satisfying the aspect ratio
    
    Notes:
    https://askubuntu.com/questions/584688/how-can-i-get-the-monitor-resolution-using-the-command-line
    http://rubular.com/
    '''
    import re
    from constants import DEFAULT, TOL
    anc_pass = []

    print(TOL)
    for anc in soup.find_all('a'):
        res = re.search('\d{4}\s*[x]\s*\d{4}', str(anc))
        if res:
            # check aspect ratio of pic
            w = int(res.group()[:4])
            h = int(res.group()[-4:])
            if abs(DEFAULT - w / h) < TOL:
                anc_pass.append(anc)

In [11]:
def save_imgs(urls):
    import shutil
    
    for i, url in enumerate(urls, 1):
        response = requests.get(url, stream=True)
        with open('img{}.png'.format(i), 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)

In [ ]:
def set_wall(folder):
    '''
    Selects and assigns a random wallpaper from the given folder.
    
    Notes:
    https://askubuntu.com/questions/66914/how-to-change-desktop-background-from-command-line-in-unity
    '''
    import os
    import subprocess as sp
    from random import choice
    
    os.chdir(folder)
    wall = choice(os.listdir())
    try:
        sp.check_output(['gsettings', 'set', 'org.gnome.desktop.background', 'picture-uri', 'file://{}'.format(wall)])
    except sp.CalledProcessError as e:
        print("An error occured while trying to change your wallpaper.\n"
             "Please ensure you have 'gsettings' installed.")
    